In [51]:
import cv2
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import dlib
import numpy as np

proj_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
predictor = dlib.shape_predictor(os.path.join(proj_path, 'data/weights/shape_predictor_68_face_landmarks.dat'))
model = YOLO(os.path.join(proj_path, 'scripts/train_model/train2/weights/best.pt'))

def get_face_feature(img_name):
    img = cv2.imread(os.path.join(proj_path, 'data/photos/' + img_name))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = model(img_rgb)
    results_boxes = results[0].boxes
    x1, y1, x2, y2 = list(map(int, results_boxes.xyxy[0].tolist()))

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = dlib.rectangle(x1, y1, x2, y2)
    landmarks = predictor(gray, face)   
    aligned_face = dlib.get_face_chip(img, landmarks)
    face_recognizer = dlib.face_recognition_model_v1(os.path.join(proj_path, 'data/weights/dlib_face_recognition_resnet_model_v1.dat'))
    face_feature = face_recognizer.compute_face_descriptor(aligned_face)
    return np.array(face_feature).reshape(1, -1), results


In [ ]:
import faiss


index = faiss.IndexFlatL2(128)
index = faiss.IndexIDMap(index)

In [56]:
ff1, results = get_face_feature('two_face.jpg')
dis1, ids1 = index.search(ff1, 1)
# ff2 = get_face_feature('trump.jpg')
# dis2, ids2 = index.search(ff2, 1)
# ff3 = get_face_feature('biden.jpg')
# dis3, _ = index.search(ff3, 1)


0: 384x640 2 faces, 81.0ms
Speed: 5.0ms preprocess, 81.0ms inference, 11.0ms postprocess per image at shape (1, 3, 384, 640)


In [64]:
boxes = results[0].boxes
box = boxes[0].xyxy.shape
box

torch.Size([1, 4])